In [1]:
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import wandb
from utils import *
from model import *
from dataset import *
from tqdm import tqdm
from torch.utils.data import DataLoader

# Vocab 구축

In [2]:
#%pip install sentencepiece

In [3]:
import sentencepiece as spm

model_file = "./data/kowiki.model" 

# SentencePiece 모델 로드
sp = spm.SentencePieceProcessor()
sp.load(model_file)


True

# 학습 데이터 불러오기

In [4]:
import pandas as pd
dataset_name = 'ChatbotData_KorQuAD'
csv_path = './data/ChatbotData_KorQuAD.csv'
df = pd.read_csv(csv_path)
df.head(10)

,Q,A
0,1941년 이우가 배속된 소속은?,조선군사령부
1,1787년경 프랑스에서 거두고 있던 소득세는 무엇일까?,벵티엠
2,늑대 보호 조치와 효과적 법 집행으로 적당한 늑대 개체수를 유지하고 있는 국가는?,이스라엘
3,독도가 한국영토로 표기된 사례는 세계지도 3380건 중에 몇 건인가?,49건
4,중력과 관계된 낙차에 의해 움직여지는 수차를 무엇이라 부르는가?,중력수차
5,나랑 놀아줘,같이 놀아요.
6,친구들한테 인기 얻으려면,성격이 좋으면 인기가 있을 거예요.
7,시민단체는 정부가 기초연금에 대해 어떤 원리를 경직되게 적용한다고 보았는가?,보충성의 원리
8,한국독립당이 창당될 때 안창호 외 창당 발기인은 몇 명이었는가?,28명
9,노무현이 5공 청문회에서 명패를 던졌던 사람은?,전두환


In [5]:
#시퀀스 max 길이 찾기
seq_max_len = 0
for line in list(df['Q'].values):
    leng = len(sp.encode_as_ids(line))
    if seq_max_len < leng:
        seq_max_len = leng
print("Q seq_max_len:", seq_max_len)


for line in list(df['A'].values):
    leng = len(sp.encode_as_ids(line))
    if seq_max_len < leng:
        seq_max_len = leng
print("A seq_max_len:", seq_max_len)

#학습데이터 Vocab 적용해보기
ids_stack = []
for line in list(df['Q'][:5].values):
    pieces = sp.encode_as_pieces(line)
    ids = sp.encode_as_ids(line)
    ids += (seq_max_len-len(ids))*[0]
    ids_stack.append(ids)
    print("Original Text:", line)
    print("Tokens:", pieces)
    print("IDs:", ids)
    print()


Q seq_max_len: 78
A seq_max_len: 78
Original Text: 1941년 이우가 배속된 소속은?
Tokens: ['▁194', '1', '년', '▁이', '우', '가', '▁배', '속', '된', '▁소속', '은', '?']
IDs: [429, 3597, 3616, 8, 3679, 3599, 179, 3763, 3703, 765, 3604, 4245, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Original Text: 1787년경 프랑스에서 거두고 있던 소득세는 무엇일까?
Tokens: ['▁17', '87', '년', '경', '▁프랑스', '에서', '▁거두', '고', '▁있던', '▁소', '득', '세는', '▁무', '엇', '일', '까', '?']
IDs: [381, 3209, 3616, 3673, 542, 10, 1987, 3600, 804, 68, 4054, 1561, 108, 4491, 3620, 3794, 4245, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Original Text: 늑대 보호 조치와 효과적 법 집행으로 적당한 늑대 개체수를 유지하고 있는 국가는?
Tokens: ['▁', '늑', '대', '▁보호', '▁조', '치', '와', '▁효과', '적', '▁법', '▁집', '행', '으로', '▁

In [6]:
train_dataset = ChatBotDataset(csv_path, seq_max_len)
val_dataset = ChatBotDataset(csv_path, seq_max_len, train=False)

print("train_dataset length:", len(train_dataset))
print("val_dataset length:", len(val_dataset)) 

train_dataset length: 65007
val_dataset length: 7223


# Scaled_dot_Attention 계산 확인

In [7]:
n_dim = 3
q = torch.tensor([[[1,2,3],
                   [3,2,1],
                   [4,5,6]],
                  
                  [[3,2,2],
                   [1,1,1],
                   [5,2,4]]], dtype=torch.float32)
k = q.transpose(-1,-2)
token_ids = torch.tensor([[1,1,0], [1,0,0]])


scaled_attention = torch.matmul(q,k) / np.sqrt(n_dim)
masked_attention =  making_padding_mask(scaled_attention, token_ids)
attention_score = torch.softmax(masked_attention, dim=-1)
output = torch.matmul(attention_score, q)
print("정답:", output)


정답: tensor([[[1.1807, 2.0000, 2.8193],
         [2.8193, 2.0000, 1.1807],
         [1.1807, 2.0000, 2.8193]],

        [[3.0000, 2.0000, 2.0000],
         [3.0000, 2.0000, 2.0000],
         [3.0000, 2.0000, 2.0000]]])


In [8]:
#확인할 때 q,k,v의 가중치는 제외하고 확인해볼 것
model = ScaledDotProductAttention(3)
model(q,q,q,token_ids)

tensor([[[-0.1136, -0.9815,  2.9034],
         [ 0.0749, -1.1777,  2.9638],
         [ 0.2185, -1.3270,  3.0097]],

        [[-0.0239, -1.0855,  3.2401],
         [-0.0239, -1.0855,  3.2401],
         [-0.0239, -1.0855,  3.2401]]], grad_fn=<UnsafeViewBackward0>)

# Hyper Parameter

In [9]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

#model hyperparameter
model_name = 'transformer_chatbot'
n_seq = seq_max_len
n_vocab = sp.vocab_size()
# 논문의 절반으로 setting
n_dim = 256
n_head = 4
n_layer = 3

# dataloader hyperparameter
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#train, Loss, Optimizer hyperparameter
epochs = 100
lr = 1e-5

# Transformer Model

In [19]:
model = Transformer(n_seq, n_vocab, n_dim, n_head, n_layer, device=device).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [11]:
def train(loss_fn, optimizer, save_path, eval_step=1):
    
    wandb.init(
    project='Transformer_ChatBot',
    name = f'{model_name}_{dataset_name}',
    config={
        "architecture": model_name,
        "dataset": dataset_name,
        "batch_size" : batch_size,
        "lr": lr,
        "epochs": epochs,
        "n_seq": n_seq,
        "n_vocab": n_vocab,
        "n_dim": n_dim,
        "n_head": n_head,
        "n_layer": n_layer,
        "loss_fn": "CrossEntropyLoss",
        "optimizer": "Adam",
        })
    

    model.train()
    best_val_loss = np.inf
    # BOS 토큰 ID를 가져옵니다.
    bos_token_id = sp.bos_id()
    for epoch in tqdm(range(epochs), ascii=True, desc="epoch"):
        print()
        print(f"********** epoch{epoch+1} train start **********")
        train_loss = 0
        for idx, (q, a) in enumerate(train_dataloader):
            q, a = q.to(device), a.to(device)
            # a를 bos_a로 변경 (오른쪽 시프트)
            bos_a = shift_right(a, bos_token_id)
            
            optimizer.zero_grad()
            pred = model(q, bos_a)
            loss = loss_fn(pred.view(-1, pred.size(-1)), a.view(-1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            
            if idx % (len(train_dataloader)//3) == 0:
                print("epoch: {}, loss: {}".format(epoch+1, loss.item()))
        train_metrics = {'train_loss': train_loss/len(train_dataloader)}
        
        print("epoch: {}, train_loss: {}".format(epoch+1, train_loss/len(train_dataloader)))
        print("********** train end **********")
        wandb.log(train_metrics, step=epoch)
        print()
        if epoch % eval_step == 0:
            with torch.no_grad():
                print("********** eval start **********")
                model.eval()
                val_loss = 0
                for idx, (q, a) in enumerate(val_dataloader):
                    q, a = q.to(device), a.to(device)
                    pred = model(q, a)
                    loss = loss_fn(pred.view(-1, pred.size(-1)), a.view(-1))
                    val_loss += loss.item()
                
                if best_val_loss > val_loss:
                    best_val_loss = val_loss
                    torch.save(model.state_dict(), save_path)
                    print()
                    print("****best model saved****")
                    print()
                val_metrics = {'val_loss': val_loss/len(val_dataloader)}
                print("epoch: {}, val_loss: {}".format(epoch+1, val_loss/len(val_dataloader)))
                
                #문장 만들기
                indx = np.random.randint(0, len(q))#0~len(val_dataset) 사이의 숫자 랜덤으로 뽑기
                max_arg = torch.argmax(pred, dim=-1)
                pred_sentc = sp.DecodeIds(max_arg[indx].tolist())
                label_sentc = sp.DecodeIds(a[indx].tolist())
                q_sentc = sp.DecodeIds(q[indx].tolist())
                print("Q:", q_sentc)
                print("Pred:", pred_sentc)
                print("Label:", label_sentc)
                wandb.log(val_metrics, step=epoch)
                print("********** eval end **********")
    wandb.finish()

In [12]:
save_path = f'./weight/best_{model_name}.pt'
train(loss_fn, optimizer,save_path, eval_step=1)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dwkim8155 (boostcamp-oif). Use `wandb login --relogin` to force relogin


epoch:   0%|          | 0/100 [00:00<?, ?it/s]


**********epoch1 train start**********
epoch: 1, loss: 9.135636329650879
epoch: 1, loss: 0.882185697555542
epoch: 1, loss: 0.8259548544883728
epoch: 1, loss: 0.6149036884307861
epoch: 0, train_loss: 1.1399515186707805
**********train end**********

**********eval start**********


epoch:   1%|1         | 1/100 [03:07<5:09:42, 187.70s/it]


****best model saved****

epoch: 1, val_loss: 0.6459396887669521
Q: J. K. 롤링이 책을 집필하기 위해 자주 이용했던 곳은?
Pred: 
Label: 카페
**********eval end**********

**********epoch2 train start**********
epoch: 2, loss: 0.6310096383094788
epoch: 2, loss: 0.5644517540931702
epoch: 2, loss: 0.5706245303153992
epoch: 2, loss: 0.4819130599498749
epoch: 1, train_loss: 0.5767584623783593
**********train end**********

**********eval start**********


...

epoch: 15, val_loss: 0.7851275063194005
Q: 오스트레일리아까치는 얼마나 높은 음을 낼 수 있나?
Pred: 니이
Label: 4 옥타브 이상
**********eval end**********

**********epoch16 train start**********
epoch: 16, loss: 0.3176496922969818
epoch: 16, loss: 0.3642856478691101
epoch: 16, loss: 0.3594782054424286
epoch: 16, loss: 0.32227662205696106
epoch: 15, train_loss: 0.3336987718939781
**********train end**********

**********eval start**********


epoch:  16%|#6        | 16/100 [1:12:08<13:45:27, 589.62s/it]

epoch: 16, val_loss: 0.7996084136245525
Q: 말제르브가 정치에 개입하기 시작한 해는?
Pred: 권
Label: 1771년
**********eval end**********

**********epoch17 train start**********
epoch: 17, loss: 0.26528438925743103


epoch:  16%|#6        | 16/100 [1:12:23<6:20:01, 271.44s/it] 


KeyboardInterrupt: 

In [22]:
#저장된 best 가중치 불러오기
save_path = f'./weight/best_{model_name}.pt'
state_dict = torch.load(save_path)
model.load_state_dict(state_dict)

model.eval()
prompt = "내일 뭐해?"
output = generate_sentc(model, sp, n_seq, prompt, device)
print("입력 문장:", prompt)
print("출력 문장:", output)

입력 문장: 내일 뭐해?
출력 문장: 년
